In [2]:
import jax
import jax.numpy as jnp
import haiku as hk

import sinusoid_regression_dataset

In [3]:
dataset = sinusoid_regression_dataset.SinusoidRegression(32, 5, 666)
def net(x):
    x = hk.nets.MLP((32, 32, 32, 32, 2))(x)
    return jnp.split(x, -1)

In [4]:
import models

init, apply = hk.without_apply_rng(hk.transform(net))
n_particles = 10
init = jax.vmap(init, (0, None))
apply = jax.vmap(apply, (0, None))
seed_sequence = hk.PRNGSequence(666)
example = next(dataset.train_set)[0]
hyper_prior_particles = init(jnp.asarray(seed_sequence.take(n_particles)), example)
hyper_prior = models.ParamsMeanField(hyper_prior_particles)
prior_particles = init(jnp.asarray(seed_sequence.take(n_particles)), example)
prior = models.ParamsMeanField(hyper_prior_particles)


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/home/yarden/miniconda3/envs/jax-ml/lib/python3.10/site-packages/haiku/_src/base.py:515: UserWarning: Explicitly requested dtype float64 requested in zeros is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  param = init(shape, dtype)


In [5]:
import optax
import pacoh_nn

opt = optax.flatten(optax.adam(2e-3))
opt_state = opt.init(prior_particles)
pacoh_nn.meta_train(dataset.train_set, apply, hyper_prior, prior, opt, opt_state, 1, 10)

TypeError: Argument 'mlp/~/linear_0' of type <class 'str'> is not a valid JAX type.